In [5]:
!pip install -U sentence-transformers datasets torch transformers adapters evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 14.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 39.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 141.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 126.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 24.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 59.2 M

In [6]:
from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset
from transformers import AutoTokenizer, AutoConfig, TrainingArguments
import torch
import csv
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'sentence_transformers'

In [5]:
eclipse_url = "https://github.com/logpai/bughub/raw/refs/heads/master/EclipsePlatform/"
firefox_url = "https://raw.githubusercontent.com/logpai/bughub/refs/heads/master/Firefox/"
jdt_url = "https://raw.githubusercontent.com/logpai/bughub/refs/heads/master/JDT/"
mozilla_url = "https://github.com/logpai/bughub/raw/refs/heads/master/MozillaCore/"
thunderbird_url = "https://raw.githubusercontent.com/logpai/bughub/refs/heads/master/Thunderbird/"

def create_datasets(url, bugs):
    dataset = load_dataset("csv", data_files = url + bugs)
    train = load_dataset("csv", data_files = url + "train.csv")
    test = load_dataset("csv", data_files = url + "test.csv")
    dataset_df = dataset['train'].to_pandas()
    train_df = train['train'].to_pandas()
    test_df = test['train'].to_pandas()    
    train_data = train_df.merge(dataset_df, on="Issue_id", how="left")
    train_data = pd.DataFrame(train_data).values.tolist()
    test_data = test_df.merge(dataset_df, on="Issue_id", how="left")
    test_data = pd.DataFrame(test_data).values.tolist()
    issue_dict = {issue["Issue_id"]: issue for issue in dataset["train"].to_list()}
    return train_data, test_data, issue_dict

eclipse_train, eclipse_test, eclipse_dict = create_datasets(eclipse_url, "eclipse_platform.zip")
firefox_train, firefox_test, firefox_dict = create_datasets(firefox_url, "mozilla_firefox.zip")
jdt_train, jdt_test, jdt_dict = create_datasets(jdt_url,"eclipse_jdt.csv")
mozilla_train, mozilla_test, mozilla_dict = create_datasets(mozilla_url, "mozilla_core.zip")
thunderbird_train, thunderbird_test, thunderbird_dict = create_datasets(thunderbird_url, "mozilla_thunderbird.csv")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
import random
from sentence_transformers import InputExample

SEED = 69  # Change this number for different results

def set_seed(seed):
    random.seed(seed)  # Python's random module
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # PyTorch CPU
    torch.cuda.manual_seed(seed)  # PyTorch GPU (if available)
    torch.cuda.manual_seed_all(seed)  # For multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False  # Avoids non-deterministic optimizations


def create_pairs(dataset, issue_dict):
    pairs = []
    for entry in dataset:
        issue_id = entry[0] 
        duplicates = entry[1]  # List of duplicate issue IDs
        
        if duplicates:
            duplicates = entry[1].split(";")
            for dup_id in duplicates:
                if int(dup_id) in issue_dict:
                    dup_id = int(dup_id)
                    bug_1 = (issue_dict[issue_id]["Component"] or "") + " " + (issue_dict[issue_id]["Title"] or "") + " " + (issue_dict[issue_id]["Description"] or "")
                    bug_2 = (issue_dict[dup_id]["Component"] or "") + " " + (issue_dict[dup_id]["Title"] or "") + " " + (issue_dict[dup_id]["Description"] or "")
                    pairs.append(InputExample(
                        texts = [bug_1, bug_2],
                        label = 1
                    ))

            # Create negative samples (random non-duplicate issues)
            for _ in range(len(duplicates)):  # Create equal number of negative pairs
                random_id = random.choice(list(issue_dict.keys()))
                if str(random_id) not in duplicates and random_id != issue_id:
                    bug_1 = (issue_dict[issue_id]["Component"] or "") + " " + (issue_dict[issue_id]["Title"] or "") + " " + (issue_dict[issue_id]["Description"] or "")
                    bug_2 = (issue_dict[random_id]["Component"] or "") + " " + (issue_dict[random_id]["Title"] or "") + " " + (issue_dict[random_id]["Description"] or "")
                    pairs.append(InputExample(
                        texts = [bug_1, bug_2],
                        label = 0
                    ))
    
    return pairs

from concurrent.futures import ThreadPoolExecutor

datasets = [
    ("eclipse_train", eclipse_train[0:100], eclipse_dict),
    ("eclipse_test", eclipse_test[0:20], eclipse_dict),
    ("firefox_train", firefox_train[0:100], firefox_dict),
    ("firefox_test", firefox_test[0:20], firefox_dict),
    ("jdt_train", jdt_train[0:100], jdt_dict),
    ("jdt_test", jdt_test[0:20], jdt_dict),
    ("mozilla_train", mozilla_train[0:100], mozilla_dict),
    ("mozilla_test", mozilla_test[0:20], mozilla_dict),
    ("thunderbird_train", thunderbird_train[0:100], thunderbird_dict),
    ("thunderbird_test", thunderbird_test[0:20], thunderbird_dict),
]

results = {}

def process_dataset(name, data, dictionary):
    return name, create_pairs(data, dictionary)

with ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_dataset, name, data, dictionary): name for name, data, dictionary in datasets}
    
    for future in futures:
        name, result = future.result()
        results[name] = result

# Assign results to variables dynamically
eclipse_train_samples = results["eclipse_train"]
eclipse_test_samples = results["eclipse_test"]
firefox_train_samples = results["firefox_train"]
firefox_test_samples = results["firefox_test"]
jdt_train_samples = results["jdt_train"]
jdt_test_samples = results["jdt_test"]
mozilla_train_samples = results["mozilla_train"]
mozilla_test_samples = results["mozilla_test"]
thunderbird_train_samples = results["thunderbird_train"]
thunderbird_test_samples = results["thunderbird_test"]

train_samples = eclipse_train_samples + firefox_train_samples + jdt_train_samples + mozilla_train_samples + thunderbird_train_samples
test_samples =  eclipse_test_samples + firefox_test_samples + jdt_test_samples + mozilla_test_samples + thunderbird_test_samples

In [47]:
# NORMAL FINE-TUNING
# Adjust Learning Rate, Weight decay

from torch.utils.data import DataLoader
from sentence_transformers import losses

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=16, worker_init_fn=lambda _: set_seed(SEED))
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
#if torch.cuda.device_count()>1:
#    print (f"Using{torch.cuda.device_count()} GPUs!")
#    model = torch.nn.DataParallel(model)
#model.to(device)
train_loss = losses.CosineSimilarityLoss(model)
num_epochs = 2
warmup_steps = int(0.05 * len(train_dataloader) * num_epochs)
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    use_amp=True,
    show_progress_bar=True,
    optimizer_params={"lr": 2e-5, "weight_decay": 0.01}  # Adjust these values
)

Detected kernel version 4.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


In [34]:
#ADAPTER TRAINING

import torch
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, losses
import adapters

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=16, worker_init_fn=lambda _: set_seed(SEED))
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Add an adapter for fine-tuning
adapter_name = "pfeiffer_adapter"

# Get the underlying Hugging Face transformer model
transformer_model = model._first_module().auto_model 

# Add and activate adapter
adapters.init(transformer_model)
transformer_model.add_adapter(adapter_name, config="pfeiffer", set_active=True)
transformer_model.train_adapter(adapter_name)

train_loss = losses.CosineSimilarityLoss(model)
num_epochs = 2
warmup_steps = int(0.05 * len(train_dataloader) * num_epochs)
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    use_amp=True,
    show_progress_bar=True,
    optimizer_params={"lr": 2e-5, "weight_decay": 0.01}  # Learning rate and weight decay
)


Detected kernel version 4.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


In [8]:
#LAYER SPECIFIC FINE TUNING

import torch
from sentence_transformers import SentenceTransformer, losses
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=16, worker_init_fn=lambda _: set_seed(SEED))
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Optionally freeze lower layers (e.g., layers 0 to 5)
for name, param in model.named_parameters():
    if '0' <= name.split('.')[1] <= '5':  # Assuming the layers you want to freeze are in range 0 to 5
        param.requires_grad = False

train_loss = losses.CosineSimilarityLoss(model)
num_epochs = 2
warmup_steps = int(0.05 * len(train_dataloader) * num_epochs)

# Train the model with layer-specific fine-tuning
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    use_amp=True,
    show_progress_bar=True,
    optimizer_params={"lr": 2e-5, "weight_decay": 0.01}  # Adjust these values
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Detected kernel version 4.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


In [28]:
from sentence_transformers import util
import torch

# Function to evaluate test samples
def evaluate_model(model, test_samples):
    TP, FP, FN = 0, 0, 0
    total = len(test_samples)

    for sample in test_samples:
        text1, text2 = sample.texts
        label = sample.label  # Ground truth (1 for duplicate, 0 for non-duplicate)

        # Compute embeddings
        emb1 = model.encode(text1, convert_to_tensor=True)
        emb2 = model.encode(text2, convert_to_tensor=True)

        # Compute cosine similarity
        similarity = util.pytorch_cos_sim(emb1, emb2).item()

        # Threshold (adjust based on your dataset)
        prediction = 1 if similarity > 0.5 else 0

        # Compare with ground truth
        # Count TP, FP, FN
        if prediction == 1 and label == 1:
            TP += 1  # True Positive
        elif prediction == 1 and label == 0:
            FP += 1  # False Positive
        elif prediction == 0 and label == 1:
            FN += 1  # False Negative


    # Compute metrics
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    f2_score = (5 * precision * recall) / ((4 * precision) + recall) if (precision + recall) > 0 else 0 
    
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1_score:.4f}")
    print(f"F2-score: {f2_score:.4f}") 

# Run evaluation
evaluate_model(model, test_samples)

Precision: 0.9524
Recall: 0.7018
F1-score: 0.8081
F2-score: 0.7407
